In [8]:
import TokenClassificationTrainer
import utils
import numpy as np
from evaluate import load as load_metric

## import the confusions matrix
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import pickle as pkl


In [5]:
task = "ner"
model_name = "xlm-mlm-17-1280"  # "bert-base-multilingual-cased" or "xlm-mlm-17-1280"
seed = 79002
save_name = 'baseline'+ ".seed-" + str(seed)
batch_size = 32


test_trainer = TokenClassificationTrainer.TokenClassificationTrainer(task=task, model_name=model_name, save_name=save_name, batch_size=batch_size, label_all_tokens=True, file_paths={"train" : "data/datasets/baseline/en_ewt_nn_train_newsgroup_and_weblogs.conll", "validation" : "data/datasets/NoSta-D/NER-de-train.tsv"})
test_trainer.set_trainer(use_old = True)

Some weights of the model checkpoint at xlm-mlm-17-1280 were not used when initializing XLMForTokenClassification: ['pred_layer.proj.weight', 'pred_layer.proj.bias']
- This IS expected if you are initializing XLMForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMForTokenClassification were not initialized from the model checkpoint at xlm-mlm-17-1280 and are newly initialized: ['classifier.bias', 'classifier.weight', 'transformer.position_ids']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
names=[
                    "O",
                    "B-PER",
                    "I-PER",
                    "B-ORG",
                    "I-ORG",
                    "B-LOC",
                    "I-LOC",
                    "B-MISC",
                    "I-MISC"
]

metric = load_metric("seqeval")


In [6]:
# get the saved model and make predictions
predictions, labels = test_trainer.predict("data/datasets/hungarian/hungarian_test.tsv")
predictions = np.argmax(predictions, axis=2)

true_predictions = [
    [names[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

true_labels = [
    [names[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results
    

  1%|          | 5/750 [00:56<2:40:40, 12.94s/it]                  

KeyboardInterrupt: 

In [ ]:
true_p_f = [pred for preds in true_predictions for pred in preds]
true_l_f = [pred for preds in true_labels for pred in preds]

In [1]:
conf_mat = confusion_matrix(true_l_f, true_p_f, labels=names, normalize = 'true')


In [4]:
fig, ax = plt.subplots(figsize=(10, 10))
disp = ConfusionMatrixDisplay(conf_mat)

disp.plot(ax = plt.gca())
ax.set_xticks(range(len(names)),names)
ax.set_yticks(range(len(names)),names)

plt.show()